# Trading compute for memory in PyTorch models using Checkpointing

Given the fact that deep learning explorations are bottlenecked by device memory constraints, we implemented a gradient checkpointing technique described in [1] and using this technique, we can achieve 4x-10x larger models ranging across convolutional networks, recurrent networks and medical imaging applications. The implementations of memory optimized versions of models along with their baselines is available [here](https://github.com/prigoyal/pytorch_memonger/tree/master/models). The results of checkpointing are below:

<img src="results.png">

As we see from the above results, for ResNet-1001 model, baseline implementation can only fit 15 images and for better packing and layers optimization, we use minibatch size of 8 per run and do 4 runs to simulate minibatch size 32 training. But in the checkpointed model, we can fit ~52 images in minibatch and hence use 32 images in each run. As evident from the results, per iteration <i> speed improves by ~10% </i> and the model accuracy will be better because of stable batch normalization layers. 

###  Gradient Checkpointing

Gradient Checkpointing technique reorders the forward and reverse computation of the model so as to reduce the maximal length of the autograd tape. This is achieved by dividing the model or parts of the model in various segments and executing the segments without taping them in the forward pass i.e. their taping is delayed until the backward pass. This results in not storing the activations of those segments in memory. During the backward pass, since we need activations to compute the gradients, the forward pass is done again on the segment and the pass is taped. In this way, at any given time, the tape length is the maximum of tape length of various segments and not the sum.

##  Using Checkpointing for PyTorch models

Now in this tutorial, we will see how to use checkpointing on various models in PyTorch.

<b>NOTE:</b> This tutorial needs PyTorch master branch which can be installed by following instructions [here](https://github.com/pytorch/pytorch#from-source)

In [2]:
# some standard imports
import torch
from torch.autograd import Variable

### Checkpointing sequential models

First, let's create a simple Sequential model and checkpoint it. We can also verify that the checkpointing doesn't 
change the value of gradients or the activations.

In [ ]:
# import the checkpoint API
from torch.utils.checkpoint import checkpoint_sequential
import torch.nn as nn

# create a simple Sequential model
model = nn.Sequential(
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 20),
    nn.ReLU(),
    nn.Linear(20, 5),
    nn.ReLU()
)

# create the model inputs
input_var = Variable(torch.randn(1, 100), requires_grad=True)

# set the number of checkpoint segments
segments = 2

# get the modules in the model. These modules should be in the order
# the model should be executed
modules = [module for k, module in model._modules.items()]

# now call the checkpoint API and get the output
out = checkpoint_sequential(modules, segments, input_var)

# run the backwards pass on the model. For backwards pass, for simplicity purpose, 
# we won't calculate the loss and rather backprop on out.sum()
model.zero_grad()
out.sum().backward()

# now we save the output and parameter gradients that we will use for comparison purposes with
# the non-checkpointed run.
output_checkpointed = out.data.clone()
grad_checkpointed = {}
for name, param in model.named_parameters():
    grad_checkpointed[name] = param.grad.data.clone()

Now that we have executed the checkpointed pass on the model, let's also run the non-checkpointed model and verify that the checkpoint API doesn't change the model outputs or the parameter gradients.

In [ ]:
# non-checkpointed run of the model
original = model

# create a new variable using the same tensor data
x = Variable(input_var.data, requires_grad=True)

# get the model output and save it to prevent any modifications
out = original(x)
out_not_checkpointed = out.data.clone()

# calculate the gradient now and save the parameter gradients values
original.zero_grad()
out.sum().backward()
grad_not_checkpointed = {}
for name, param in model.named_parameters():
    grad_not_checkpointed[name] = param.grad.data.clone()

Now that we have done the checkpointed and non-checkpointed pass of the model and saved the output and parameter gradients, let's compare their values

In [ ]:
# compare the output and parameters gradients
self.assertEqual(out_checkpointed, out_not_checkpointed)
for name in grad_checkpointed:
    self.assertEqual(grad_checkpointed[name], grad_not_checkpointed[name])

So, from this example, we can see that it's very easy to use checkpointing on Sequential models and that the checkpoint API doesn't alter any data. The Checkpoint API implementation is based on autograd and hence there is no need for explicitly specifying what the execution of backwards should look like. For more examples on checkpointing sequential models like ResNet and DenseNet, see the examples [here](https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/resnet_new.py) and [here](https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/densenet_new.py).

<b>TIP: </b> As you might have notices in the above examples of Pre-activation ResNet/ DenseNet, the resnet block is directly appended to ```self.features``` and similarly Denselayer is directly appended to ```self.features```. Doing this allows for better granular checkpointing otherwise, wrapping Denselayers in DenseBlock (as is done in baseline), or resnetBlocks in ResNet layers would limit checkpointing to DenseBlock or ResNet layer which consumes more memory.

<b> IMPORTANT NOTE</b>: The list of modules should strictly define the model execution and it shouldn't be arbitrarily arranged. 

### Checkpointing non-sequential models
So far, we have seen how to do checkpointing on Sequential models like ResNet, DenseNet etc. for which we can easily extract the modules and call the ```checkpoint_sequential```. But there are other models like LSTM etc. which can't truly use ```checkpoint_sequential```. Now, let's see how we can use checkpointing in such cases. We will take an example for RNN model which is available [here](https://github.com/pytorch/examples/blob/master/word_language_model/model.py). For simplicity, the weight inits and hidden unit inits have been omitted from definition below.

In [ ]:
# some standard imports
import torch.nn as nn
from torch.autograd import Variable

# define the model 
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        self.nhid = nhid
        self.nlayers = nlayers

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

In the above LSTM based model, we can optimize memory by checkpointing ```self.rnn()``` execution by segmenting the BPTT sequence length. In order to do this, we will use ```checkpoint()``` API call which requires passing a ```run_function``` that describes what should the checkpoint API run in forward pass for a given segment of the model. So let's define such a run function and modify the model as below:

In [ ]:
# some standard imports
import torch.nn as nn
from torch.autograd import Variable

# import the checkpoint API 
import torch.utils.checkpoint as checkpoint

# modified model definition to use checkpointing
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        self.nhid = nhid
        self.nlayers = nlayers

    # define a run_function that returns a function which takes the inputs variables as 
    # arguments, unpacks those inputs, perform the computation and returns the results. 
    # Note that the start, end are not inuts to custom_forward function below.    
    def run_function(self, start, end):
        def custom_forward(*inputs):
            output, hidden = self.rnn(
                inputs[0][start:(end+1)], (inputs[1], inputs[2])
            )
            return output, hidden[0], hidden[1]
        return custom_forward
    
    def forward(self, input, hidden, segments):
        emb = self.drop(self.encoder(input))
        # checkpoint self.rnn() computation
        output = []
        segment_size = len(modules) // segments
        for start in range(0, segment_size * (segments - 1), segment_size):
            end = start + segment_size - 1
            # Note that if there are multiple inputs, we pass them as as is without 
            # wrapping in a tuple etc.
            out = checkpoint.checkpoint(
                self.run_function(start, end), emb, hidden[0], hidden[1]
            )
            output.append(out[0])
            hidden = (out[1], out[2])
        out = checkpoint.checkpoint(
            self.run_function(end + 1, len(modules) - 1), emb, hidden[0], hidden[1]
        )
        output.append(out[0])
        hidden = (out[1], out[2])
        
        output = torch.cat(output, 0)
        hidden = (out[1], out[2])
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

In summary, for checkpointing any part of the model, you can define a ```run_function``` which describes what should the checkpoint API run in the forward pass. Please see the above example and comments on the code.

<b> NOTE: </b> In case of checkpointing, if all the inputs don't require grad but the outputs do, then if the inputs are passed as is, the output of Checkpoint will be variable which don't require grad and autograd tape will break there. To get around, you can pass a dummy input which requires grad but isn't necessarily used in computation.

### Checkpointing any module

If you want to checkpoint only a certain module in your network, it's very easy to checkpoint it. Let's create a dummy model comprising of many layers and we want to checkpoint one module in it.

In [ ]:
# some standard imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable, Function
import torch.utils.checkpoint as checkpoint

class ConvBNReLU(nn.Module):
    
    def __init__(self, in_planes, out_planes):
        
        super(ConvBNReLU, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu1 = nn.ReLU(inplace=True)
    
    def forward(self, x):
        out = self.relu1(self.bn1(self.conv1(x)))
        return out

class DummyNet(nn.Module):
    def __init__(self):
        super(DummyNet, self).__init__()
        self.features = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1', nn.BatchNorm2d(16)),
            ('relu1', nn.ReLU(inplace=True)),
        ]))

        # The module that we want to checkpoint
        self.module = ConvBNReLU(16, 64) 
        
        self.final_module = ConvBNReLU(64, 64)
    
    def custom(self, module):
        def custom_forward(*inputs):
            inputs = module(inputs[0])
            return inputs
        return custom_forward
    
    def forward(self, x):
        out = self.features(x)
        out = checkpoint.checkpoint(self.custom(self.module), out)
        out = self.final_module(out)
        return out

# Handling a few special layers in checkpointing

While using checkpointing, here are a few useful things that should be followed in order to avoid any issues with the output and grad values. 

### Dropout Layer
The checkpointing API runs the forward pass twice on the checkpointed segment of the model: once during the forward pass which is untaped, and second times during the backward pass where the forward pass is taped. Since dropout drops values arbitrarily, <b> avoid </b> having dropout in the checkpointed model part otherwise it will result in incorrect output. Instead, split the checkpointing into two parts: before and after the dropout layer

### Batch Normalization layer
Batch normalization layer maintains the running mean and variance stats depending on the current minibatch and everytime a forward pass is run, the stats are updated based on the ```momentum``` value. In checkpointing, running the forward pass twice on a model segment in the same iteration will result in updating mean and stats value. In order to avoid this, use the ```new_momentum = sqrt(momentum)``` as the momentum value. The mathematical justification is as follows:

In [ ]:
original_rm, original_nm = running_mean, new_mean

# Originally, we have a running_mean to start from
momemtum = 0.9
# running mean after the first forward pass. We want to match this 
# when we use checkpointing
running_mean1 = original_nm * 0.1 + original_rm * 0.9

# With checkpointing, we use momentum = sqrt(0.9) and make two passes
momentum = sqrt(0.9)
runing_mean = original_nm * (1 - sqrt(0.9)) + original_rm * sqrt(0.9)      # Eq (1) 
# Note that in the second forward pass, since the inputs are same, the 
# new_mean value doesn't change
running_mean2 = original_nm * (1 - sqrt(0.9)) + running_mean * sqrt(0.9)   # Eq (2) 

#Substitue value of running_mean from Eq(1) to Eq(2)
assert running_mean2 == running_mean1

### Decoder layer in Word language model
In the Pytorch example for [word language model](https://github.com/pytorch/examples/blob/master/word_language_model/model.py#L20), the decoder is a single fully connected layer that operates on the entire sequence length and batch size. This layer is followed by Cross Entropy loss calculation which first calculates the softmax on the output of decoder. Now, as the sequence length is increases, the decoder activation size and softmax output size increases as well and is very large in size (almost entire memory consumption is because of decoder layer). In order to get around, we can split the decoder layer in segments, calculate the loss on each segment and do the loss backward on each segment. This saves tremendous amount of memory. For reference see [this](https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/word_language_model_new.py)

## References

[1]. Siskind, Jeffrey Mark, and Barak A. Pearlmutter. "Divide-and-Conquer Checkpointing for Arbitrary Programs with No User Annotation." arXiv preprint arXiv:1708.06799 (2017).

[2]. T. Chen, B. Xu, C. Zhang, and C. Guestrin. Training deep nets with sublinear memory cost.
arXiv preprint arXiv:1604.06174, 2016

[3]. Gomez, A. N., Ren, M., Urtasun, R., & Grosse, R. B. (2017). The reversible residual network: Backpropagation without storing activations. In Advances in Neural Information Processing Systems (pp. 2211-2221).